
### **Goal:**
To build a hierarchical image classification model using MobileNetV2 for level_1 and level_2 classification tasks.

---

### **Steps Overview:**
1. **Load and Prepare the Dataset**
2. **Build the MobileNetV2 Model**
3. **Compile and Train the Model**
4. **Evaluate the Model**
5. **Visualize Results**

---

# 1. Load and Prepare the dataset 


In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator 